In [ ]:
import pygame
pygame.init()
WIDTH, HEIGHT = 960, 540
screen = pygame.display.set_mode((WIDTH, HEIGHT)) #kích thước màn hình
pygame.display.set_caption("Lets go Stage 1 nào!!") # tên hiện lên trên màn hình
running = True
while running: #loop các sự kiện trong game
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
pygame.quit() # out game khi kết thúc vòng lặp
#Basic settings: 

FPS = 60
TILE = 48

# Vật lý đơn giản
GRAVITY = 0.6
MAX_FALL = 14
PLAYER_SPEED = 4.2
JUMP_VEL = -12
COYOTE_MS = 120  # nhảy "cứu cánh" sau khi rời đất ~0.12s

# Màu sắc
BG = (20, 24, 28)
WHITE = (240, 240, 240)
GREY = (120, 128, 140)
GREEN = (80, 200, 120)
RED = (220, 60, 80)
YELLOW = (250, 210, 90)
CYAN = (100, 210, 230)
BROWN = (120, 90, 60)
ORANGE = (240, 150, 60)

#============= Using class to creat world ================
class Player:
    def __init__(self, x, y):
        # tạo hình chữ nhật đại diện cho nhân vật (tọa độ x, y và kích thước 20x30)
        self.rect = pygame.Rect(x, y, 20, 30)
        self.vel_x = 0              # vận tốc theo trục ngang (x)
        self.vel_y = 0              # vận tốc theo trục dọc (y)
        self.on_ground = False      # kiểm tra xem nhân vật đang đứng trên mặt đất hay không
        self.health = 3             # số mạng hoặc máu của nhân vật

    def handle_input(self, keys):
        """xử lý phím điều khiển người chơi"""
        self.vel_x = 0              # mỗi frame reset lại vận tốc ngang để tránh trượt
        if keys[pygame.K_a]:        # nếu nhấn phím A
            self.vel_x = -3         # di chuyển sang trái (tốc độ = 3)
        if keys[pygame.K_d]:        # nếu nhấn phím D
            self.vel_x = 3          # di chuyển sang phải
        if keys[pygame.K_w] and self.on_ground:  # nếu nhấn W và đang ở trên đất
            self.vel_y = -8         # nhảy lên (âm nghĩa là đi lên trên)
            self.on_ground = False  # lúc này không còn đứng trên đất nữa

    def apply_gravity(self):
        """áp dụng trọng lực để nhân vật rơi dần xuống"""
        if self.vel_y < MAX_FALL:   # nếu chưa đạt tốc độ rơi tối đa
            self.vel_y += GRAVITY   # tăng dần vận tốc rơi (mô phỏng trọng lực)

    def update(self, platforms):
        """cập nhật vị trí của nhân vật và kiểm tra va chạm"""
        # di chuyển theo trục x trước (trái-phải)
        self.rect.x += self.vel_x
        self.check_collision_x(platforms)  # kiểm tra va chạm ngang

        # sau đó di chuyển theo trục y (rơi, nhảy)
        self.rect.y += self.vel_y
        self.check_collision_y(platforms)  # kiểm tra va chạm dọc

class Platform:
    def __init__(self, x, y, w, h, moving=False, speed=0):
        # tạo hình chữ nhật cho platform với kích thước (w, h)
        self.rect = pygame.Rect(x, y, w, h)
        self.moving = moving              # cho biết platform có di chuyển không
        self.speed = speed       # tốc độ di chuyển của platform (nếu có)
        self.triggered = False   # dùng để biết người chơi đã chạm vào chưa?
        self.direction = 1       # hướng di chuyển: 1 = xuống, -1 = lên
        self.start_y = y         # lưu lại vị trí ban đầu (để platform dao động quanh đó)

# gai nhấp nhô
class Spike:
    def __init__(self, x, y, up_time=1000, down_time=1000):
        # mỗi gai là 1 ô vuông có kích thước bằng TILE
        self.rect = pygame.Rect(x, y, TILE, TILE)
        self.up_time = up_time           # thời gian gai ở trạng thái trồi lên (ms)
        self.down_time = down_time       # thời gian gai ở trạng thái hạ xuống (ms)
        self.timer = 0                   # bộ đếm thời gian
        self.active = False              # ban đầu gai ở trạng thái hạ xuống

    def update(self, dt):
        """bật / tắt gai theo thời gian, dt là thời gian trôi qua giữa 2 frame"""
        self.timer += dt                 # tăng bộ đếm theo thời gian
        if self.active and self.timer >= self.up_time:   # nếu gai đang trồi lên quá lâu
            self.active = False          # hạ xuống
            self.timer = 0               # reset bộ đếm
        elif not self.active and self.timer >= self.down_time:  # nếu đang hạ xuống quá lâu
            self.active = True           # trồi lên
            self.timer = 0               # reset bộ đếm

# ốc sên di chuyển qua lại
class MovingSnail:
    def __init__(self, x, y, path_length, speed):
        self.rect = pygame.Rect(x, y, TILE, TILE)     # tạo hình chữ nhật cho ốc sên
        self.start_x = x                              # lưu vị trí bắt đầu
        self.path_length = path_length                # độ dài quãng đường mà ốc sên đi được
        self.speed = speed                            # tốc độ di chuyển
        self.direction = 1                            # hướng di chuyển: 1 = phải, -1 = trái
        self.damage = 1                               # sát thương gây ra khi người chơi chạm

# laser bắn các hướng 
class Laser:
    def __init__(self, x, y, direction='right', active_after=1000):
        # laser thường được đặt cố định ở một vị trí
        self.rect = pygame.Rect(x, y, TILE, TILE)
        self.direction = direction        # hướng bắn: 'left', 'right', 'up', hoặc 'down'
        self.active_after = active_after  # thời gian bật / tắt xen kẽ (ms)
        self.timer = 0                    # bộ đếm thời gian
        self.active = False               # ban đầu laser tắt
        self.damage = 1                   # sát thương laser gây ra khi bật

# Tạo world
class World:
    def __init__(self):
        # danh sách chứa các đối tượng có trong màn chơi
        self.platforms = []    # danh sách các platform (nền)
        self.spikes = []       # danh sách các gai
        self.snails = []       # danh sách các ốc sên
        self.lasers = []       # danh sách các tia laser

